In [1]:
#API key - eq63r7svaqsta7wm_6bzrykdx2zj4jfnh
from san import *
import san
from decimal import Decimal
import pandas as pd
import datetime

In [2]:
#Getting all slugs/tickers/names/marketsegment/supply and dropping all with missing or 0 supply
data = san.get("projects/all")
data = data.dropna()
data['totalSupply'] = data['totalSupply'].astype(float)
data = data[data['totalSupply'] != 0]

In [3]:
#limiting only to cryptocurrency
crypto = data[data["marketSegment"] == "Cryptocurrency"]
crypto = crypto.sort_values(by = ["totalSupply"], ascending = False)
crypto

,marketSegment,name,slug,ticker,totalSupply
1748,Cryptocurrency,Dimecoin,dimecoin,DIME,5.559981e+11
1181,Cryptocurrency,Mooncoin,mooncoin,MOON,2.274909e+11
1854,Cryptocurrency,NewYorkCoin,newyorkcoin,NYC,1.437611e+11
1707,Cryptocurrency,Dogecoin,dogecoin,DOGE,1.291413e+11
1769,Cryptocurrency,eXPerience Chain,experience-chain,XPC,1.132155e+11
...,...,...,...,...,...
1442,Cryptocurrency,ParallelCoin,parallelcoin,DUO,3.147186e+05
347,Cryptocurrency,Unobtanium,unobtanium,UNO,2.033284e+05
238,Cryptocurrency,Wrapped Bitcoin,wrapped-bitcoin,WBTC,1.459507e+05
1761,Cryptocurrency,Amoveo,amoveo,VEO,6.983818e+04


In [10]:
def metriclist(slug):
    metric = "price_usd" #change metric here. Use san.available_metrics() or san.available_metrics_for_slug(slug)
    fullname = metric + "/" + slug 
    metrics = san.get(
    fullname,
    from_date="2018-04-01",
    to_date="2021-04-01",
    interval="1d"
    )
    newname = slug+"_"+metric
    finaldf = metrics.rename(columns={"value": newname})
    return(finaldf)

In [11]:
#Change metric and slug fullname = metric + slug
#Running through all cryptocurrencies and adding price and daily returns
count = 0
metric_df = pd.DataFrame()
for i in crypto['slug'].tolist():
    holddf = metriclist(i)
    if count == 0:
        metric_df = holddf
    else:
        metric_df = pd.merge(allpdret, holddf, how = "left", on = "datetime")
    count = count+1